In [1]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import tabulate
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# pip install transformers==4.45.2 



d:\code\dl-hw5\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [2]:
dataset = load_dataset('dair-ai/emotion')
print(dataset)

labels = dataset["train"].features["label"].names
print(labels)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})
['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [3]:
model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [4]:
def tokenize_data(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Токенизация всех данных
tokenized_dataset = dataset.map(tokenize_data, batched=True)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [5]:
def compute_metrics(eval_pred):
    logits, true_labels = eval_pred  # Переименовали переменную
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(true_labels, predictions),
        **classification_report(
            true_labels, 
            predictions,
            target_names=labels,  # Используем глобальный список названий
            output_dict=True,
            zero_division=0
        )["macro avg"]
    }


### Full finetuning

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels)
)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {trainable_params:,}")

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=1000,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # max_steps=1000,
    num_train_epochs=3,
    logging_steps=200,
    weight_decay=0.01,
    metric_for_best_model="f1-score",
    logging_dir="./logs",
    report_to="none",
    load_best_model_at_end=True,
    save_total_limit=2,
    # lr_scheduler_type="cosine",
    # warmup_steps=500
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))
trainer.train()

results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))

# Training time: 31m


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total trainable parameters: 109,486,854


100%|██████████| 125/125 [00:23<00:00,  5.27it/s]


+-----------------------------+------------+
| Метрика                     |   Значение |
+=============================+============+
| eval_loss                   |     1.8294 |
+-----------------------------+------------+
| eval_model_preparation_time |     0.0000 |
+-----------------------------+------------+
| eval_accuracy               |     0.1230 |
+-----------------------------+------------+
| eval_precision              |     0.0759 |
+-----------------------------+------------+
| eval_recall                 |     0.1425 |
+-----------------------------+------------+
| eval_f1-score               |     0.0772 |
+-----------------------------+------------+
| eval_support                |  2000.0000 |
+-----------------------------+------------+
| eval_runtime                |    29.0365 |
+-----------------------------+------------+
| eval_samples_per_second     |    68.8790 |
+-----------------------------+------------+
| eval_steps_per_second       |     4.3050 |
+---------

  7%|▋         | 200/3000 [01:38<22:20,  2.09it/s]

...

{'eval_loss': 0.16834279894828796, 'eval_model_preparation_time': 0.0, 'eval_accuracy': 0.9405, 'eval_precision': 0.9206623386735681, 'eval_recall': 0.9136635887827942, 'eval_f1-score': 0.9165278983047488, 'eval_support': 2000.0, 'eval_runtime': 22.6711, 'eval_samples_per_second': 88.218, 'eval_steps_per_second': 5.514, 'epoch': 3.0}


100%|██████████| 3000/3000 [30:21<00:00,  1.65it/s]


{'train_runtime': 1821.7417, 'train_samples_per_second': 26.348, 'train_steps_per_second': 1.647, 'train_loss': 0.23278478399912517, 'epoch': 3.0}


100%|██████████| 125/125 [00:22<00:00,  5.59it/s]

+-----------------------------+------------+
| Метрика                     |   Значение |
+=============================+============+
| eval_loss                   |     0.1777 |
+-----------------------------+------------+
| eval_model_preparation_time |     0.0000 |
+-----------------------------+------------+
| eval_accuracy               |     0.9275 |
+-----------------------------+------------+
| eval_precision              |     0.8902 |
+-----------------------------+------------+
| eval_recall                 |     0.8693 |
+-----------------------------+------------+
| eval_f1-score               |     0.8788 |
+-----------------------------+------------+
| eval_support                |  2000.0000 |
+-----------------------------+------------+
| eval_runtime                |    22.7870 |
+-----------------------------+------------+
| eval_samples_per_second     |    87.7690 |
+-----------------------------+------------+
| eval_steps_per_second       |     5.4860 |
+---------

### Linear probing

In [ ]:
from torch import nn

from torch import nn
from transformers import BertConfig

class TransformerClassifierHead(nn.Module):
    def __init__(self, hidden_size, num_labels, num_layers=1, dropout=0.1):
        super().__init__()
        self.config = BertConfig(
            hidden_size=hidden_size,
            num_attention_heads=12,
            intermediate_size=3072,
            num_hidden_layers=num_layers,
            hidden_dropout_prob=dropout
        )
        
        self.transformer = nn.TransformerEncoder(
            encoder_layer=nn.TransformerEncoderLayer(
                d_model=hidden_size,
                nhead=self.config.num_attention_heads,
                dim_feedforward=self.config.intermediate_size,
                dropout=dropout,
                activation="gelu",
                batch_first=True
            ),
            num_layers=num_layers
        )
        
        self.pooler = nn.Linear(hidden_size, hidden_size)
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, hidden_states):
        if hidden_states.dim() == 2:
            hidden_states = hidden_states.unsqueeze(1)  # [batch, 1, hidden]
        
        transformer_out = self.transformer(hidden_states)  # [batch, 1, hidden]
        
        pooled = self.tanh(self.pooler(transformer_out[:, 0, :]))  # [batch, hidden]
        pooled = self.dropout(pooled)
        
        return self.classifier(pooled)

# Модифицируем модель BERT
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels)
)

# Замораживаем все слои BERT
for param in model.parameters():
    param.requires_grad = False

# Добавляем кастомную голову
hidden_size = model.config.hidden_size  # 768 для bert-base
num_labels = len(labels)                        
model.classifier = TransformerClassifierHead(hidden_size, num_labels)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {trainable_params:,}")

# Проверяем, что обучаются только параметры классификатора
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Обучается: {name}")

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=1000,
    learning_rate=1e-4, # Больше lr, так как обучаем только голову
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # max_steps=10000,
    num_train_epochs=10,
    logging_steps=1000,
    weight_decay=0.01,
    metric_for_best_model="f1-score",
    logging_dir="./logs",
    report_to="none",
    load_best_model_at_end=True,
    save_total_limit=2,
    lr_scheduler_type="cosine",
    # warmup_steps=500
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))

# Training time: 37m

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total trainable parameters: 7,683,078
Обучается: classifier.transformer.layers.0.self_attn.in_proj_weight
Обучается: classifier.transformer.layers.0.self_attn.in_proj_bias
Обучается: classifier.transformer.layers.0.self_attn.out_proj.weight
Обучается: classifier.transformer.layers.0.self_attn.out_proj.bias
Обучается: classifier.transformer.layers.0.linear1.weight
Обучается: classifier.transformer.layers.0.linear1.bias
Обучается: classifier.transformer.layers.0.linear2.weight
Обучается: classifier.transformer.layers.0.linear2.bias
Обучается: classifier.transformer.layers.0.norm1.weight
Обучается: classifier.transformer.layers.0.norm1.bias
Обучается: classifier.transformer.layers.0.norm2.weight
Обучается: classifier.transformer.layers.0.norm2.bias
Обучается: classifier.pooler.weight
Обучается: classifier.pooler.bias
Обучается: classifier.classifier.weight
Обучается: classifier.classifier.bias


 10%|█         | 1000/10000 [03:13<29:05,  5.16it/s]

{'loss': 1.5412, 'grad_norm': 4.264253616333008, 'learning_rate': 9.755282581475769e-05, 'epoch': 1.0}
































































































































                                           
                                                    
 10%|█         | 1000/10000 [03:35<29:05,  5.16it/s]


...

{'eval_loss': 1.248012900352478, 'eval_accuracy': 0.527, 'eval_precision': 0.45005689601292165, 'eval_recall': 0.3218399257842363, 'eval_f1-score': 0.3024922018707328, 'eval_support': 2000.0, 'eval_runtime': 24.2452, 'eval_samples_per_second': 82.49, 'eval_steps_per_second': 5.156, 'epoch': 10.0}



100%|██████████| 10000/10000 [36:49<00:00,  4.53it/s]


{'train_runtime': 2209.9339, 'train_samples_per_second': 72.4, 'train_steps_per_second': 4.525, 'train_loss': 1.3958416015625, 'epoch': 10.0}


100%|██████████| 125/125 [00:23<00:00,  5.26it/s]

+-------------------------+------------+
| Метрика                 |   Значение |
+=========================+============+
| eval_loss               |     1.2296 |
+-------------------------+------------+
| eval_accuracy           |     0.5315 |
+-------------------------+------------+
| eval_precision          |     0.4409 |
+-------------------------+------------+
| eval_recall             |     0.3190 |
+-------------------------+------------+
| eval_f1-score           |     0.3073 |
+-------------------------+------------+
| eval_support            |  2000.0000 |
+-------------------------+------------+
| eval_runtime            |    23.9604 |
+-------------------------+------------+
| eval_samples_per_second |    83.4710 |
+-------------------------+------------+
| eval_steps_per_second   |     5.2170 |
+-------------------------+------------+
| epoch                   |    10.0000 |
+-------------------------+------------+


### Prompt tuning

In [14]:
from peft import (
    PromptTuningConfig,
    get_peft_model,
    TaskType
)
from transformers import AutoModelForSequenceClassification

peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=20,  # Длина мягкого промпта
    token_dim=768,          # Размерность эмбеддингов BERT
    prompt_tuning_init="TEXT",
    prompt_tuning_init_text="Classify the emotion in the text:",
    base_model_name_or_path="bert-base-uncased",
    tokenizer_name_or_path="bert-base-uncased"
)

model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-uncased",
    num_labels=len(labels),
    return_dict=True
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()  # Обучаемые параметры: ~0.1%


training_args = TrainingArguments(
    output_dir="./peft_results",
    learning_rate=1e-4,           # Выше обычного для промптов
    per_device_train_batch_size=32,
    num_train_epochs=5,          # Нужно больше эпох
    logging_steps=100,
    save_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 15,360 || all params: 109,502,214 || trainable%: 0.0140


  0%|          | 0/5000 [1:47:34<?, ?it/s]

 20%|█▉        | 496/2500 [08:47<27:05,  1.23it/s]

{'loss': 1.8323, 'grad_norm': 0.20117510855197906, 'learning_rate': 9.6e-05, 'epoch': 0.2}



 20%|█▉        | 496/2500 [10:08<27:05,  1.23it/s]

...


 20%|█▉        | 496/2500 [19:44<27:05,  1.23it/s]

{'loss': 1.677, 'grad_norm': 0.5308024883270264, 'learning_rate': 6.400000000000001e-05, 'epoch': 1.8}



 20%|█▉        | 496/2500 [21:04<27:05,  1.23it/s] 

{'loss': 1.6627, 'grad_norm': 0.4204711318016052, 'learning_rate': 6e-05, 'epoch': 2.0}


KeyboardInterrupt: 

### Lora

In [ ]:

from peft import (
    LoraConfig, 
    TaskType, 
    get_peft_model
)
from transformers import AutoModelForSequenceClassification

lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        r=8,                # Ранг адаптеров
        lora_alpha=16,      # Коэффициент масштабирования
        lora_dropout=0.1,   # Дропаут для регуляризации
        target_modules=["query", "value"],  # Слои для применения LoRA
        bias="none"
    )


model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-uncased",
    num_labels=len(labels)
)

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()


training_args = TrainingArguments(
    output_dir=f"./lora_results",
    learning_rate=3e-4,
    per_device_train_batch_size=32,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    logging_steps=200
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))

# Training time: 36m

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 299,526 || all params: 109,786,380 || trainable%: 0.2728


  8%|▊         | 200/2500 [02:48<31:26,  1.22it/s]

{'loss': 1.349, 'grad_norm': 2.0876481533050537, 'learning_rate': 0.000276, 'epoch': 0.4}


 16%|█▌        | 400/2500 [05:33<28:45,  1.22it/s]

...

{'loss': 0.1844, 'grad_norm': 2.3966174125671387, 'learning_rate': 1.1999999999999999e-05, 'epoch': 4.8}


100%|██████████| 2500/2500 [35:20<00:00,  1.28it/s]





















































































































































































































































                                                   

100%|██████████| 2500/2500 [35:48<00:00,  1.28it/s]


100%|██████████| 2500/2500 [35:48<00:00,  1.16it/s]


{'eval_loss': 0.1654692143201828, 'eval_accuracy': 0.938, 'eval_precision': 0.9193392246932405, 'eval_recall': 0.9178323360827396, 'eval_f1-score': 0.9184753633939998, 'eval_support': 2000.0, 'eval_runtime': 27.0808, 'eval_samples_per_second': 73.853, 'eval_steps_per_second': 9.232, 'epoch': 5.0}
{'train_runtime': 2148.5027, 'train_samples_per_second': 37.235, 'train_steps_per_second': 1.164, 'train_loss': 0.40780635681152344, 'epoch': 5.0}


100%|██████████| 250/250 [00:26<00:00,  9.35it/s]

+-------------------------+------------+
| Метрика                 |   Значение |
+=========================+============+
| eval_loss               |     0.1978 |
+-------------------------+------------+
| eval_accuracy           |     0.9220 |
+-------------------------+------------+
| eval_precision          |     0.8764 |
+-------------------------+------------+
| eval_recall             |     0.8873 |
+-------------------------+------------+
| eval_f1-score           |     0.8813 |
+-------------------------+------------+
| eval_support            |  2000.0000 |
+-------------------------+------------+
| eval_runtime            |    26.8544 |
+-------------------------+------------+
| eval_samples_per_second |    74.4760 |
+-------------------------+------------+
| eval_steps_per_second   |     9.3090 |
+-------------------------+------------+
| epoch                   |     5.0000 |
+-------------------------+------------+


### Вывод
По результатам видно, что full finetuning и lora показывают наилучшие результаты.